# PyTorch to Jax/Haiku Weight Transfer (POC)

Proof of concept transferring the weights of Werner Duvaud's networks to a Jax implementation.

## Imports and instantiation

In [87]:
# WERNER IMPORTS
import models
from games.cartpole import MuZeroConfig

# JAX IMPORTS
import jax
import jax.numpy as jnp
import haiku as hk
import numpy as np

import pickle

In [2]:
config = MuZeroConfig()

In [3]:
model = models.MuZeroNetwork(config)
s = model.state_dict()

In [62]:
s = {'representation_network.module.0.weight': jnp.array([[-0.3506,  0.0419, -0.4661,  0.1495],
        [-0.3790,  0.3501, -0.0726, -0.2318],
        [ 0.4639, -0.2674, -0.1594,  0.0798],
        [-0.3707,  0.0974, -0.1867, -0.3819],
        [ 0.0205, -0.1866,  0.3080, -0.1279],
        [ 0.3898,  0.1907, -0.1453,  0.2172],
        [-0.0826, -0.2179,  0.4823, -0.0043],
        [ 0.2199,  0.0094,  0.1267,  0.0637]]), 'representation_network.module.0.bias': jnp.array([-0.2076,  0.3779, -0.2294, -0.3677, -0.4350, -0.4996, -0.2542, -0.4429]), 'dynamics_encoded_state_network.module.0.weight': jnp.array([[-0.0559,  0.0137,  0.2536, -0.0065, -0.1450,  0.3137, -0.1632, -0.0156,
         -0.2441,  0.3076],
        [-0.0867,  0.1092, -0.2273,  0.1115,  0.1422,  0.0609, -0.0515,  0.2555,
          0.2003, -0.2189],
        [ 0.2730, -0.1407,  0.0028, -0.1780, -0.0351, -0.1259, -0.1297,  0.3114,
          0.2170, -0.0824],
        [ 0.2266,  0.0572, -0.0689,  0.1373, -0.2304,  0.2930, -0.1496, -0.2402,
          0.1274,  0.3037],
        [-0.2448, -0.1454,  0.1497,  0.1618, -0.2779, -0.1059, -0.1865, -0.2381,
          0.2124, -0.2016],
        [-0.1938,  0.0596,  0.0382, -0.2457, -0.0388, -0.0330,  0.2450,  0.1380,
         -0.0954, -0.0357],
        [ 0.1776,  0.0213,  0.2830,  0.1132,  0.0942, -0.1542,  0.3092, -0.0580,
         -0.2454, -0.0257],
        [ 0.0790, -0.3112,  0.3039, -0.0582,  0.1758,  0.1379,  0.2323, -0.0741,
          0.2113,  0.2262],
        [-0.2614,  0.0941, -0.0938,  0.1773, -0.0727,  0.2873, -0.0536, -0.1623,
         -0.1282,  0.1771],
        [ 0.2829,  0.2035, -0.0407, -0.1483,  0.1923,  0.1867, -0.0622,  0.3087,
         -0.2906, -0.2777],
        [-0.1866, -0.2336,  0.1308, -0.0453, -0.0474,  0.1994, -0.2737,  0.2393,
         -0.1020, -0.2272],
        [-0.1080, -0.1141,  0.1839,  0.0006, -0.1014, -0.2631, -0.2634,  0.3116,
         -0.3072,  0.0891],
        [-0.2299, -0.1013, -0.2473, -0.0348,  0.1152, -0.0032, -0.3047,  0.1239,
         -0.0267,  0.0303],
        [-0.2573, -0.1817, -0.0350, -0.0502,  0.1891,  0.0910,  0.2544,  0.1342,
          0.2414,  0.1097],
        [-0.0288,  0.0323, -0.2621,  0.2006,  0.0855,  0.1629, -0.2223, -0.2599,
         -0.0272,  0.0270],
        [ 0.2234,  0.0286,  0.0508,  0.1262, -0.0021,  0.0622,  0.0224,  0.1819,
         -0.1172,  0.0973]]), 'dynamics_encoded_state_network.module.0.bias': jnp.array([ 0.0219, -0.2073,  0.2734, -0.3067, -0.3148, -0.1505, -0.0262,  0.2694,
        -0.0078,  0.0842, -0.1297, -0.0441,  0.2616,  0.3148,  0.2150, -0.2879]), 'dynamics_encoded_state_network.module.2.weight': jnp.array([[ 0.0811,  0.1843, -0.0983,  0.2112,  0.1574,  0.0599,  0.1098,  0.0353,
          0.0864,  0.2383,  0.2321, -0.0030,  0.1342, -0.0064,  0.0674,  0.2075],
        [-0.0915,  0.1567, -0.0344, -0.1147,  0.1507,  0.2108, -0.1727, -0.1546,
         -0.1703,  0.1824,  0.0836,  0.1690, -0.0502, -0.2027, -0.1392,  0.2099],
        [ 0.0502, -0.2198, -0.1322, -0.1142,  0.1830,  0.0666,  0.1375, -0.0531,
          0.0277, -0.1427,  0.1859, -0.1622, -0.1196,  0.2320,  0.0329,  0.1360],
        [ 0.0592,  0.1847,  0.1927, -0.1229, -0.1410,  0.2326,  0.0601, -0.1729,
          0.1747, -0.1435, -0.0066,  0.2085, -0.0236, -0.0983,  0.2364,  0.1767],
        [ 0.1620,  0.1123, -0.1150,  0.0706, -0.0770,  0.1756,  0.0746,  0.1220,
          0.0545,  0.2041,  0.2306,  0.1456,  0.0033,  0.2433, -0.2369, -0.1413],
        [ 0.2428,  0.2246,  0.1674, -0.1763, -0.0885,  0.1519,  0.0140, -0.1222,
          0.0452, -0.2331, -0.0399, -0.0110, -0.1394,  0.1352,  0.0534,  0.1053],
        [ 0.2441, -0.0507, -0.0787,  0.0731,  0.1386,  0.1581, -0.1167,  0.2478,
          0.2423,  0.1698, -0.2348, -0.0894, -0.2110, -0.0090, -0.2457, -0.0021],
        [-0.1673,  0.0712, -0.2447, -0.0163,  0.1067,  0.0555,  0.0525,  0.0156,
         -0.1142,  0.1073, -0.2380,  0.2474, -0.0367, -0.2208, -0.2093,  0.1900]]), 'dynamics_encoded_state_network.module.2.bias': jnp.array([-0.1161,  0.2401, -0.1185, -0.2307,  0.2390, -0.1411,  0.0208, -0.0858]), 'dynamics_reward_network.module.0.weight': jnp.array([[-0.0672,  0.0590,  0.1732, -0.2128,  0.1921, -0.3216, -0.0280, -0.2409],
        [-0.1763,  0.1240,  0.3387, -0.3434, -0.2181, -0.0431,  0.0069, -0.1583],
        [-0.1952,  0.1732,  0.1974, -0.2550, -0.1388,  0.0581,  0.2023, -0.0086],
        [-0.2956,  0.1182,  0.3303, -0.1634, -0.0005, -0.0740,  0.1271,  0.0051],
        [ 0.1733, -0.2934,  0.2828, -0.3444,  0.3136,  0.0015, -0.3002, -0.2085],
        [-0.2608,  0.2017, -0.3117,  0.0400, -0.2991,  0.3077,  0.2452,  0.2196],
        [ 0.0188,  0.2116, -0.0721, -0.1162,  0.3377, -0.0680,  0.0256, -0.2203],
        [-0.1676, -0.0169, -0.2031,  0.1561, -0.1158,  0.0724,  0.2040, -0.0240],
        [ 0.2114,  0.1683, -0.3051, -0.1452, -0.0263,  0.3311, -0.3018,  0.0961],
        [ 0.3237, -0.0835,  0.1812, -0.2944, -0.0461,  0.1630, -0.2735, -0.2905],
        [-0.1255, -0.1222, -0.2593,  0.0134, -0.1390,  0.1140, -0.1467, -0.2345],
        [-0.0951, -0.0238, -0.1415, -0.2558, -0.0880, -0.2588, -0.1312, -0.0290],
        [-0.0298,  0.3251,  0.2610, -0.2856,  0.1977,  0.0377,  0.2203,  0.3072],
        [ 0.3440,  0.2652,  0.1974, -0.3529,  0.0505, -0.1833,  0.2340, -0.3080],
        [-0.0911,  0.0338, -0.2897,  0.2257, -0.2224,  0.2447,  0.1917,  0.0107],
        [-0.1597,  0.0182, -0.0366, -0.3476,  0.1204,  0.0902, -0.3341,  0.3297]]), 'dynamics_reward_network.module.0.bias': jnp.array([-0.0965,  0.0588, -0.2602, -0.1621,  0.0579, -0.1968, -0.2130, -0.0878,
        -0.3194,  0.3530, -0.1314, -0.1390,  0.0529,  0.3526,  0.0458,  0.2962]), 'dynamics_reward_network.module.2.weight': jnp.array([[-1.1733e-01, -1.7171e-01,  2.3485e-01, -1.1267e-01,  1.0610e-05,
          1.6636e-01, -2.0820e-01, -2.4953e-01, -8.4100e-02,  1.7835e-01,
          1.3633e-01, -2.3070e-03, -1.6641e-01,  2.1942e-01,  1.7289e-01,
         -1.5245e-03],
        [ 1.1726e-01, -2.4570e-01, -8.8032e-02,  2.4200e-01,  8.7195e-02,
         -6.5871e-02, -2.3970e-01,  1.0926e-02,  1.3790e-01,  3.8941e-02,
          1.5282e-01, -2.1673e-01, -1.7481e-01,  1.0582e-01,  4.0595e-02,
         -2.2949e-01],
        [-1.3576e-01,  2.0808e-01,  1.5728e-01, -2.0707e-01,  1.5042e-01,
         -6.9661e-02, -1.7258e-01,  1.4869e-01, -2.3614e-01, -3.4556e-02,
          1.6263e-01, -2.0161e-01, -2.0571e-01, -3.6219e-02,  9.3397e-02,
          1.8556e-01],
        [-2.4789e-01,  1.9737e-01,  1.4781e-02, -1.1783e-01, -2.8769e-02,
         -1.9394e-01,  1.4585e-01, -2.4007e-01,  1.4390e-01, -1.6097e-01,
         -3.0948e-02, -4.6718e-02,  2.2468e-01,  7.2753e-02, -8.4777e-02,
         -1.9545e-01],
        [ 1.9189e-01,  1.9416e-01, -5.0359e-02,  8.0278e-02,  9.3848e-05,
          1.0355e-01,  2.3642e-01, -6.2404e-02, -9.5068e-02,  3.1892e-02,
         -5.3143e-03,  2.2222e-01,  2.3493e-03, -1.2672e-01,  2.3667e-01,
          1.6519e-01],
        [ 6.7502e-05,  3.8345e-02,  1.7072e-01,  8.3516e-02,  1.8914e-01,
         -1.0643e-01, -1.3207e-01, -1.0006e-01, -1.0212e-03,  3.6011e-02,
         -2.2283e-01, -1.9247e-01, -1.4807e-01,  1.9366e-01, -1.9136e-01,
         -8.0796e-02],
        [-3.3760e-03,  1.7318e-01, -6.0899e-02,  2.2187e-01, -1.6591e-01,
         -1.0708e-02, -7.1321e-02, -1.3721e-01, -1.4752e-01,  1.5430e-01,
          1.2353e-01,  1.6288e-01,  2.3144e-02, -1.2511e-01, -2.1917e-01,
          7.6854e-04],
        [-1.6657e-01, -2.1615e-01, -4.3875e-02, -1.2127e-01, -1.6127e-01,
          2.4997e-01,  3.9603e-02, -4.7864e-02,  2.4738e-01, -2.1565e-02,
          1.0320e-01, -2.4444e-01,  9.5432e-03, -3.9440e-02,  2.4710e-02,
         -1.7127e-01],
        [ 2.4928e-02,  1.6867e-01,  9.3142e-02, -2.2409e-01, -2.3045e-01,
          1.7068e-01, -2.3655e-01,  1.3549e-01,  1.3653e-01, -2.0225e-01,
         -1.5625e-01,  1.4167e-01, -1.2345e-01, -1.5407e-01, -9.7583e-02,
          1.6660e-01],
        [-2.2005e-01,  4.8523e-02, -1.2936e-01,  2.1488e-01,  7.7741e-02,
         -2.3475e-01, -1.7203e-01,  1.9046e-01,  9.5499e-02, -2.0859e-01,
          2.2184e-01, -1.9518e-01, -1.0647e-01,  2.2915e-01,  1.5588e-01,
         -1.7641e-02],
        [-1.3467e-01, -7.3634e-02,  3.1147e-02,  9.4285e-02,  1.5700e-01,
         -7.2142e-02,  1.6534e-01, -1.6767e-01, -6.6262e-02,  1.0374e-01,
          4.3617e-02, -1.6648e-01, -1.1132e-02, -5.0623e-02, -2.0718e-01,
          2.4050e-01],
        [ 1.0880e-01, -4.5782e-02, -1.7972e-01,  1.4073e-02,  1.0060e-01,
          1.4837e-01, -1.8196e-01, -2.2306e-03,  1.7834e-01, -1.5695e-01,
          8.8891e-02,  1.4707e-01,  1.5844e-01,  2.2540e-02, -1.2229e-01,
         -1.8747e-01],
        [-1.4811e-01,  3.0859e-02, -1.3637e-01, -1.1519e-01, -9.4373e-02,
          6.3136e-02, -1.1423e-02, -2.2137e-01, -7.0044e-02, -2.0579e-02,
          2.4972e-01,  2.1066e-01, -1.7214e-01,  2.5368e-02, -6.9327e-02,
          9.3162e-02],
        [-2.4496e-01, -2.3260e-01, -7.4793e-03, -1.7304e-01,  8.4996e-02,
          1.2495e-01, -2.0984e-01,  1.2511e-01, -2.0542e-01, -1.2143e-01,
          1.3831e-01, -2.4335e-01,  3.8048e-02, -6.6714e-02,  4.1304e-02,
          6.9145e-02],
        [ 1.9952e-01,  1.1150e-01,  1.6135e-02,  1.4582e-01,  1.0771e-01,
         -7.8934e-02, -7.7621e-02, -1.8719e-01,  1.9978e-01, -1.4032e-01,
         -1.7708e-01,  1.8915e-01, -1.8040e-01,  1.6576e-01, -1.4574e-01,
          9.6225e-02],
        [ 2.3907e-01,  2.1488e-02, -1.5685e-01,  1.1776e-01, -2.1161e-01,
         -8.4022e-02, -1.6880e-01,  1.4822e-01,  2.1059e-01, -1.1893e-01,
          1.6456e-01,  6.7845e-02, -3.7658e-02, -1.3651e-01,  1.9336e-01,
          1.9173e-01],
        [-1.8640e-01, -2.0991e-01, -1.0416e-01, -1.7926e-01, -7.0047e-02,
         -2.1542e-01, -1.1344e-02,  1.1900e-01,  2.4157e-01, -2.3573e-01,
         -2.0184e-01,  1.6093e-01, -9.4085e-02,  1.5267e-01, -2.1713e-01,
          1.5518e-01],
        [-2.4877e-01,  1.3528e-03,  1.8537e-01,  1.6738e-01,  1.3269e-01,
          1.9863e-02, -2.4564e-01, -9.9725e-02, -1.1623e-01,  1.6963e-01,
         -1.3040e-02, -6.1580e-02, -6.0246e-02, -1.5627e-01,  2.2519e-02,
          7.7357e-02],
        [-2.1022e-01, -1.1694e-01, -2.4365e-02,  1.5477e-01, -2.1913e-01,
          1.2359e-01,  6.8757e-02, -1.7097e-01,  1.8061e-01,  1.3383e-02,
          4.6601e-02,  2.1440e-01,  1.9591e-01,  1.8877e-01, -8.4448e-03,
         -2.3499e-01],
        [ 2.2741e-01, -2.3928e-01, -2.4878e-01,  1.8760e-01,  2.4461e-02,
         -1.2340e-01, -3.1669e-02,  2.4820e-01, -6.9294e-02,  3.3330e-02,
         -1.2146e-01,  3.2416e-02,  9.2532e-02, -2.3552e-01,  1.3045e-01,
          3.0556e-02],
        [ 2.1902e-01,  1.8638e-01,  2.0112e-01,  2.1886e-01,  1.0490e-01,
         -1.8942e-01, -3.6989e-03,  1.0723e-02,  2.2903e-01, -9.1677e-02,
          1.1082e-01, -1.4619e-01,  1.0873e-01,  2.3931e-01,  2.3925e-01,
         -1.8487e-01]]), 'dynamics_reward_network.module.2.bias': jnp.array([ 0.0728, -0.2371, -0.0564,  0.0329, -0.1790,  0.1781, -0.1864, -0.1719,
         0.0970,  0.0080,  0.1335,  0.1816, -0.1034, -0.1482,  0.2165, -0.2282,
         0.1603,  0.1996, -0.1173,  0.0543,  0.0895]), 'prediction_policy_network.module.0.weight': jnp.array([[ 0.0672,  0.2385, -0.0892, -0.0815, -0.0266,  0.0302, -0.2006, -0.2909],
        [ 0.0918, -0.2497, -0.2864,  0.0697, -0.0340,  0.3275, -0.1064, -0.3389],
        [ 0.2042, -0.3156, -0.0917,  0.3100,  0.1506,  0.0799, -0.1005,  0.0037],
        [-0.2791,  0.0927,  0.1362, -0.1330, -0.2990, -0.2311,  0.0728, -0.0544],
        [ 0.2743, -0.2443,  0.0245,  0.0727, -0.1149,  0.0519, -0.3101,  0.0973],
        [-0.3416, -0.1812, -0.0961,  0.1539,  0.2849,  0.0124,  0.0352,  0.0888],
        [ 0.3529, -0.3341, -0.3407,  0.0452,  0.2999, -0.1639, -0.3225, -0.1289],
        [ 0.2788, -0.0669,  0.0066,  0.0740, -0.1848,  0.0565, -0.1857,  0.1943],
        [-0.0924,  0.2700, -0.1300, -0.3110, -0.1865,  0.2959, -0.0929,  0.3316],
        [ 0.1667, -0.3484, -0.0542, -0.3292,  0.3456,  0.0550,  0.2551, -0.1476],
        [-0.0638,  0.3429,  0.2631, -0.1538, -0.2129,  0.3476,  0.1228, -0.0578],
        [ 0.0466,  0.2855,  0.1303,  0.0581,  0.3252, -0.0743, -0.1611, -0.1806],
        [-0.2155, -0.1544, -0.1825, -0.3531,  0.0481, -0.2576, -0.0105,  0.1433],
        [-0.0596,  0.0008,  0.1892,  0.3217, -0.3354, -0.1624,  0.0215,  0.3356],
        [-0.0767,  0.1259,  0.0786,  0.2221,  0.1367, -0.1061, -0.0294, -0.0754],
        [-0.1218,  0.2991,  0.1925, -0.2629, -0.0286, -0.2500, -0.1533,  0.0948]]), 'prediction_policy_network.module.0.bias': jnp.array([ 0.3502, -0.1541, -0.3042, -0.1033,  0.3358,  0.0787,  0.0452, -0.2941,
        -0.2440, -0.2644,  0.0068,  0.1407, -0.3529, -0.1030, -0.1939, -0.2562]), 'prediction_policy_network.module.2.weight': jnp.array([[ 0.2334,  0.2470, -0.0822,  0.0943,  0.1041, -0.0490,  0.1148,  0.1666,
          0.2062, -0.0114, -0.0785,  0.0030, -0.1915,  0.0226,  0.1750, -0.0249],
        [-0.1780,  0.1414,  0.2298,  0.0647,  0.0870,  0.1928, -0.0901,  0.0097,
         -0.1962,  0.0059, -0.1664, -0.1352, -0.2272,  0.1205,  0.1084,  0.0946]]), 'prediction_policy_network.module.2.bias': jnp.array([ 0.1445, -0.0943]), 'prediction_value_network.module.0.weight': jnp.array([[-0.3206,  0.0901, -0.2454, -0.0570, -0.1085,  0.0765,  0.1496,  0.0853],
        [-0.2605,  0.2633, -0.0819, -0.2277, -0.1680,  0.3412, -0.2701,  0.0053],
        [-0.0318, -0.1335, -0.1762,  0.2320,  0.1292,  0.0769,  0.1343, -0.0989],
        [-0.0762, -0.0373, -0.0298, -0.0085,  0.0944,  0.3171,  0.3279, -0.2440],
        [ 0.0065,  0.0583,  0.2058, -0.0649, -0.0882,  0.2205,  0.0652,  0.2774],
        [ 0.1739, -0.2984, -0.0815,  0.1275, -0.3157, -0.3485, -0.0951,  0.0351],
        [ 0.3401, -0.1137,  0.1869,  0.2420,  0.1503,  0.3423, -0.2650, -0.2799],
        [ 0.1485,  0.3267,  0.2897,  0.0207, -0.2911, -0.0419,  0.2072,  0.0542],
        [ 0.0432, -0.1854, -0.1287, -0.3054, -0.2557, -0.2311,  0.2733, -0.0811],
        [ 0.3316, -0.1648,  0.2950,  0.0018,  0.2366,  0.2790, -0.0239,  0.3440],
        [-0.2553,  0.3342, -0.1939, -0.2121,  0.2085, -0.0624, -0.1563,  0.2004],
        [-0.2391, -0.1498,  0.2948,  0.0293,  0.2239, -0.1509, -0.0116, -0.0370],
        [-0.0717, -0.2191, -0.0716,  0.1655, -0.0292,  0.2836,  0.1068, -0.0962],
        [ 0.1516,  0.0274,  0.1911,  0.1812, -0.3517,  0.2977, -0.0542,  0.3139],
        [ 0.2617,  0.1970, -0.1746,  0.1646,  0.2760,  0.2036,  0.0421,  0.0995],
        [ 0.0695,  0.1120,  0.1712,  0.0026,  0.2555,  0.3261, -0.3232,  0.2393]]), 'prediction_value_network.module.0.bias': jnp.array([ 0.1572, -0.3400,  0.1840, -0.0961,  0.3066, -0.0391, -0.3169, -0.0231,
        -0.2746, -0.1814, -0.1395, -0.2130,  0.3135,  0.2850, -0.1414,  0.0977]), 'prediction_value_network.module.2.weight': jnp.array([[ 0.0296,  0.1972, -0.0381, -0.0938,  0.0576, -0.1739, -0.1971, -0.1754,
          0.1371,  0.2344,  0.1526, -0.2004,  0.0011,  0.2073,  0.1712, -0.1358],
        [-0.2454, -0.1684,  0.1138, -0.0287, -0.1436, -0.1251, -0.2203, -0.1602,
          0.0014, -0.2170, -0.1670,  0.1966,  0.1319, -0.0552, -0.2191, -0.1885],
        [ 0.0226, -0.2431,  0.1727, -0.1597,  0.2240, -0.0777, -0.1343,  0.0108,
          0.0888, -0.1924, -0.1593,  0.0353, -0.1705, -0.1971, -0.0460,  0.1974],
        [-0.0713,  0.1714, -0.1357,  0.1418, -0.1592,  0.2244,  0.1071, -0.1241,
         -0.1615, -0.1622, -0.1029, -0.0019,  0.0097, -0.1320,  0.1214,  0.0417],
        [ 0.0253, -0.0565,  0.2427,  0.2000,  0.1703,  0.1122,  0.0644, -0.1443,
          0.1768, -0.0392, -0.0040, -0.1029,  0.0283, -0.2363,  0.1930,  0.1792],
        [-0.1895, -0.1772, -0.0349,  0.0139,  0.0369,  0.0278,  0.1170,  0.1004,
         -0.1371,  0.1263, -0.0488, -0.0824, -0.0993,  0.0107,  0.2215,  0.2485],
        [-0.2111,  0.2375, -0.0097, -0.1876,  0.0904, -0.1254, -0.1679, -0.1987,
         -0.2357, -0.2240,  0.2104, -0.1311, -0.1422,  0.1448, -0.0497,  0.1729],
        [ 0.1640, -0.0815,  0.1335, -0.1572,  0.1475, -0.2007,  0.0973, -0.1215,
          0.1264, -0.0026, -0.0864, -0.2384, -0.0155, -0.1888, -0.1652, -0.0980],
        [-0.2473, -0.0181,  0.2129, -0.0861, -0.0312, -0.0373, -0.1770,  0.1712,
         -0.1902,  0.2064,  0.1977,  0.0657,  0.1224,  0.1149, -0.1233, -0.1437],
        [ 0.1625,  0.1412,  0.0057, -0.1227, -0.2257, -0.1138,  0.1825,  0.1755,
          0.0456, -0.1250,  0.0637, -0.1258,  0.0054,  0.1488,  0.0374, -0.0626],
        [ 0.1726,  0.0642, -0.2432, -0.1804,  0.0566, -0.1549, -0.0042,  0.1186,
          0.2192, -0.0585,  0.1047,  0.1223,  0.0483,  0.0768,  0.1109,  0.0128],
        [-0.0817,  0.0950, -0.0651,  0.1351, -0.2475, -0.0200, -0.1050, -0.2047,
         -0.0152, -0.0880,  0.2330, -0.2430,  0.0860, -0.2226,  0.1821, -0.1772],
        [ 0.0056, -0.0417,  0.0845, -0.1614, -0.1250, -0.0369, -0.0387,  0.1975,
         -0.2256, -0.0819, -0.1680, -0.0782, -0.1783,  0.1968, -0.0230,  0.1313],
        [-0.1882, -0.1377,  0.1297,  0.2223, -0.2187,  0.0360, -0.2400, -0.1978,
         -0.2212,  0.1900, -0.2315, -0.2410, -0.0290,  0.2365, -0.0559, -0.0489],
        [-0.1292, -0.1661,  0.0371,  0.0646,  0.0501, -0.0279, -0.1189,  0.1005,
         -0.1990,  0.2374,  0.0430, -0.1725, -0.2176, -0.2305,  0.0870, -0.0104],
        [ 0.1562,  0.2209, -0.1704,  0.0915,  0.2420,  0.1402,  0.2060, -0.2243,
          0.2214, -0.1164, -0.2105, -0.0331, -0.1161, -0.0599, -0.2156,  0.1387],
        [-0.1071,  0.1663,  0.2433,  0.2082,  0.2328,  0.2119,  0.2092, -0.2308,
         -0.1708,  0.0576,  0.1244,  0.1131, -0.1942,  0.1120, -0.1161,  0.1574],
        [-0.0352,  0.1160, -0.1275, -0.0344,  0.0028,  0.0368,  0.1507, -0.2442,
         -0.0913, -0.2286, -0.0653,  0.1514,  0.1237, -0.0479,  0.0385,  0.0431],
        [ 0.1716,  0.2421, -0.0138, -0.0792,  0.1291, -0.0652,  0.1825, -0.0435,
         -0.1292, -0.0030, -0.2438,  0.1823,  0.0694,  0.1894,  0.1731,  0.1559],
        [ 0.1100,  0.0761,  0.1477,  0.2315,  0.0118,  0.1567,  0.1211,  0.1750,
         -0.1590,  0.0833, -0.1557, -0.0645, -0.0294,  0.2057,  0.1498,  0.0905],
        [ 0.1129, -0.2118, -0.0568,  0.0959,  0.0611,  0.1360, -0.1949,  0.2301,
         -0.1254,  0.0494,  0.1549,  0.1402, -0.0066, -0.1983, -0.0088,  0.1343]]), 'prediction_value_network.module.2.bias': jnp.array([-0.1885,  0.0334,  0.2345, -0.0409,  0.2037,  0.1107,  0.2135,  0.1017,
         0.2409, -0.0322, -0.1159,  0.0099,  0.1475, -0.0389,  0.0065,  0.1706,
         0.1101, -0.1619, -0.1882,  0.1292,  0.1512])}

## Parse the PyTorch weights

We rename it to conform to Haiku conventions and cast to `jnp.array`.

In [63]:
parsed_pytorch_weights = {}

for key, value in s.items():
    split = key.split(".")
    new_key = f"{'_'.join(split[0].split('_')[:-1])}/~/mlp/~/linear_{int(int(split[-2])/2)}"
    
    value_to_set = jnp.array(value) if split[-1][0] == "b" else jnp.array(value).T
    
    if new_key not in parsed_pytorch_weights.keys():
        parsed_pytorch_weights[new_key] = {
            split[-1][0]: value_to_set
        }
    else:
        parsed_pytorch_weights[new_key][split[-1][0]] = value_to_set

list(parsed_pytorch_weights.keys())

['representation/~/mlp/~/linear_0',
 'dynamics_encoded_state/~/mlp/~/linear_0',
 'dynamics_encoded_state/~/mlp/~/linear_1',
 'dynamics_reward/~/mlp/~/linear_0',
 'dynamics_reward/~/mlp/~/linear_1',
 'prediction_policy/~/mlp/~/linear_0',
 'prediction_policy/~/mlp/~/linear_1',
 'prediction_value/~/mlp/~/linear_0',
 'prediction_value/~/mlp/~/linear_1']

## Define the networks

In [64]:
def scale_embedding(x):
    min_encoded_state = jnp.min(x, axis=0)
    max_encoded_state = jnp.max(x, axis=0)
    scale_encoded_state = max_encoded_state - min_encoded_state
    scale_encoded_state = jax.lax.select(
        scale_encoded_state < 1e-5,
        scale_encoded_state + 1e-5,
        scale_encoded_state
    )
    x = (x - min_encoded_state) / scale_encoded_state
    return x

In [65]:
class Representation(hk.Module):
    def __init__(self):
        super().__init__()
        self.fc = hk.nets.MLP([8])
        
    def __call__(self, x):
        x = self.fc(x)
        x = jax.vmap(scale_embedding)(x)
        return x

In [66]:
class DynamicsEncodedState(hk.Module):
    def __init__(self, num_actions):
        super().__init__()
        self.num_actions = num_actions
        self.fc = hk.nets.MLP([16, 8], activation=jax.nn.elu)
        
    def __call__(self, x, action):
        one_hot_action = jax.nn.one_hot(action, self.num_actions)
        concat_embedding = jnp.concatenate((x, one_hot_action), axis=1)
        return self.fc(concat_embedding)

In [67]:
class DynamicsReward(hk.Module):
    def __init__(self):
        super().__init__()
        self.fc = hk.nets.MLP([16, 21], activation=jax.nn.elu)
        
    def __call__(self, x):
        return self.fc(x)

In [68]:
class PredictionPolicy(hk.Module):
    def __init__(self):
        super().__init__()
        self.fc = hk.nets.MLP([16, 2], activation=jax.nn.elu) # change this to 3?
        
    def __call__(self, x):
        return self.fc(x)

In [69]:
class PredictionValue(hk.Module):
    def __init__(self):
        super().__init__()
        self.fc = hk.nets.MLP([16, 21], activation=jax.nn.elu)
        
    def __call__(self, x):
        return self.fc(x)

In [70]:
def create_muzero_network(num_actions):
    """This serves as a "wrapper" around the multiple subnetworks which make up MuZero."""
    SUPPORT_SIZE = 20

    def muzero_network():
        representation = Representation()
        dynamics_state = DynamicsEncodedState(num_actions)
        dynamics_reward = DynamicsReward()
        prediction_policy = PredictionPolicy()
        prediction_value = PredictionValue()

        def init(dummy_observation, dummy_action):
            # initialise all three subnetworks
            state_embedding = representation(dummy_observation)
            policy = prediction_policy(state_embedding)
            value = prediction_value(state_embedding)

            reward = dynamics_reward(state_embedding)
            next_state_embedding = dynamics_state(state_embedding, dummy_action)
            return next_state_embedding

        return init, (representation, dynamics_state, dynamics_reward, prediction_policy, prediction_value)

    return muzero_network

def create_muzero_network_transformed(*args, **kwargs):
    return hk.multi_transform(create_muzero_network(*args, **kwargs))

We just test one here. You'd obviously want to instantiate each with the correct dummy dimensions.

In [71]:
muzero_fn = create_muzero_network_transformed(2)

dummy_obs = jnp.zeros((1, 4)) # should be (1, 4) for representation network, (1, 8) for everything else
dummy_action = jnp.zeros((1,))

rng_key = jax.random.PRNGKey(42)

params = muzero_fn.init(rng_key, dummy_obs, dummy_action)

In [72]:
representation, dynamics_state, dynamics_reward, prediction_policy, prediction_value = muzero_fn.apply

In [73]:
def dynamics(params, embedding, action):
    next_embedding = dynamics_state(params, None, embedding, action)
    reward = dynamics_reward(params, None, next_embedding)
    
    scaled_next_embedding = jax.vmap(scale_embedding)(next_embedding)
    return scaled_next_embedding, reward

## Automatic weight transfer

In [74]:
mod_params = hk.data_structures.to_mutable_dict(params)

In [75]:
# GENERAL PURPOSE WEIGHT TRANSFER (works for all networks, assuming you're using the combined state_dict)
for k, v in mod_params.items():
    mod_params[k] = parsed_pytorch_weights[k]

### Pickle the weights

In [91]:
with open("pytorch_weights", "wb") as f:
    pickle.dump(mod_params, f)

## Test network with new parameters

In [171]:
# you'd want to insert a breakpoint into Werner's model to get an input/output pair
# so you can test. our network should produce the exact same output (rounding excepted).

test_obs = jnp.array([[0.6239, 1.0000, 0.3888, 0.3551, 0.2745, 0.9821, 0.0000, 0.9711]])
test_output = muzero_fn.apply[-1](mod_params, None, test_obs)

In [172]:
# (jnp.round(test_output, 3) == jnp.round(jnp.array([[-0.0515, -0.1112,  0.0423, -0.0007, -0.0943, -0.4892,  0.1184,  0.4773,
#          -0.5282,  0.0870,  0.3374,  0.0654, -0.1027,  0.0116, -0.0109,  0.0838,
#           0.0956, -0.2483, -0.3047,  0.0704,  0.2638]]), 3)).all()

In [173]:
test_obs = jnp.array([[ 0.0307,  0.0015, -0.0309, -0.0313]])
test_output = muzero_fn.apply[0](mod_params, None, test_obs)

In [195]:
(jnp.round(test_output, 3) == jnp.round(jnp.array([[0.8770, 1.0000, 0.9396, 0.6087, 0.4098, 0.9600, 0.9619, 0.0000]]), 3)).all()

DeviceArray(True, dtype=bool)

In [78]:
# next_embedding = dynamics_state(
#     mod_params,
#     None,
#     jnp.array([[0.3247, 1.0000, 0.3195, 0.1484, 0.0573, 0.0000, 0.2517, 0.0550]]),
#     jnp.array([1])
# )

next_embedding, reward = dynamics(
    mod_params,
    jnp.array([[0.3247, 1.0000, 0.3195, 0.1484, 0.0573, 0.0000, 0.2517, 0.0550]]),
    jnp.array([1])
)

In [79]:
(jnp.round(next_embedding, 3) == jnp.round(
    jnp.array([[0.0000, 0.1661, 0.2077, 0.0404, 1.0000, 0.3590, 0.9643, 0.0802]]), 3)).all()

DeviceArray(True, dtype=bool)

In [86]:
(jnp.round(reward, 2) == jnp.round(
    jnp.array([[ 0.1648, -0.2865,  0.0963, -0.0423, -0.1397,  0.2844, -0.1599, -0.4252,
         -0.0741,  0.0564,  0.2437,  0.0509, -0.0720, -0.1388,  0.2437, -0.3246,
          0.1336,  0.2252, -0.2652,  0.0564,  0.0692]]), 2)).all()

DeviceArray(True, dtype=bool)

## Manual weight transfer

You probably don't need this, but it's here just in case.

In [14]:
# mod_params['prediction_value/~/mlp/~/linear_0']['w'] = temp_p['module.0.weight'].T
# mod_params['prediction_value/~/mlp/~/linear_0']['b'] = temp_p['module.0.bias']
# mod_params['prediction_value/~/mlp/~/linear_1']['w'] = temp_p['module.2.weight'].T
# mod_params['prediction_value/~/mlp/~/linear_1']['b'] = temp_p['module.2.bias']

In [15]:
# mod_params['prediction_policy/~/mlp/~/linear_0']['w'] = temp_p['module.0.weight'].T
# mod_params['prediction_policy/~/mlp/~/linear_0']['b'] = temp_p['module.0.bias']
# mod_params['prediction_policy/~/mlp/~/linear_1']['w'] = temp_p['module.2.weight'].T
# mod_params['prediction_policy/~/mlp/~/linear_1']['b'] = temp_p['module.2.bias']

In [16]:
# mod_params['dynamics_state/~/mlp/~/linear_0']['w'] = temp_p['module.0.weight'].T
# mod_params['dynamics_state/~/mlp/~/linear_0']['b'] = temp_p['module.0.bias']
# mod_params['dynamics_state/~/mlp/~/linear_1']['w'] = temp_p['module.2.weight'].T
# mod_params['dynamics_state/~/mlp/~/linear_1']['b'] = temp_p['module.2.bias']

# mod_params['dynamics_reward/~/mlp/~/linear_0']['w'] = temp_p['module.0.weight'].T
# mod_params['dynamics_reward/~/mlp/~/linear_0']['b'] = temp_p['module.0.bias']
# mod_params['dynamics_reward/~/mlp/~/linear_1']['w'] = temp_p['module.2.weight'].T
# mod_params['dynamics_reward/~/mlp/~/linear_1']['b'] = temp_p['module.2.bias']

In [17]:
# mod_params['representation/~/mlp/~/linear_0']['w'] = jnp.array([[ 0.4536,  0.3991, -0.0396,  0.2076],
#         [-0.2074,  0.3541,  0.3898, -0.1555],
#         [ 0.3405, -0.0368, -0.0834,  0.2472],
#         [-0.3188,  0.4464,  0.4505, -0.0251],
#         [-0.2798, -0.2960,  0.2525,  0.1830],
#         [ 0.4745, -0.3207,  0.0425, -0.3299],
#         [ 0.4757, -0.1136, -0.0694,  0.4792],
#         [ 0.1024,  0.0567,  0.2998, -0.2345]]).T
# mod_params['representation/~/mlp/~/linear_0']['b'] = jnp.array([ 0.4359,  0.4771,  0.2069, -0.0961, -0.0662, -0.2580, -0.0164, -0.2669])